In [1]:
import tensorflow as tf

from common import DataOwner

import tf_encrypted as tfe
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD

/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_features = 10
training_set_size = 2000
test_set_size = 100
batch_size = 100
steps_per_epoch = (training_set_size // batch_size)
epochs = 20

In [3]:
data_owner = DataOwner('data-owner',
                       num_features,
                       training_set_size,
                       test_set_size,
                       batch_size)

x_train, y_train = tfe.define_private_input(
    data_owner.player_name,
    data_owner.provide_training_data)

x_test, y_test = tfe.define_private_input(
    data_owner.player_name,
    data_owner.provide_testing_data)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def evaluate(model, sess, x_test, y_test):
    y_hat = model(x_test)
    y_hat_np, y_test_np = sess.run([y_hat.reveal(), y_test.reveal()])
    y_hat_tf = tf.convert_to_tensor(y_hat_np)
    y_test_tf = tf.convert_to_tensor(y_test_np)
    
    correct_prediction = tf.equal(tf.round(y_hat_tf), y_test_tf)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    print("Test Accuracy", sess.run(accuracy, tag='evaluate'))

## Define Model

In [5]:
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, num_features]))
model.add(tfe.keras.layers.Activation('sigmoid'))

In [6]:
with tfe.Session() as sess:
    sess.run([tf.global_variables_initializer(), 
              data_owner.initializer], tag='init')
    
    opt = SGD(lr=0.01)
    model.compile(opt, BinaryCrossentropy())
    model.fit(x_train, y_train, epochs=epochs, steps_per_epoch=steps_per_epoch)
    
    evaluate(model, sess, x_test, y_test) 

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2', 'data-owner']
INFO:tf_encrypted:Epoch 1/20


2000/2000 [==============================] - 2s 981us/step - loss: 0.5988


INFO:tf_encrypted:Epoch 2/20


2000/2000 [==============================] - 0s 75us/step - loss: 0.5605


INFO:tf_encrypted:Epoch 3/20


...

2000/2000 [==============================] - 0s 76us/step - loss: 0.3428


INFO:tf_encrypted:Epoch 19/20


2000/2000 [==============================] - 0s 76us/step - loss: 0.3305


INFO:tf_encrypted:Epoch 20/20


2000/2000 [==============================] - 0s 77us/step - loss: 0.3338
Test Accuracy 0.96
